In [1]:
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS
import altair as alt

In [3]:
team_files = [
    "Athletic_Club.csv",
    "Atlético_Madrid.csv",
    "Barcelona.csv",
    "Celta_Vigo.csv",
    "Deportivo_Alavés.csv",
    "Espanyol.csv",
    "Getafe.csv",
    "Girona.csv",
    "Las_Palmas.csv",
    "Leganés.csv",
    "Mallorca.csv",
    "Osasuna.csv",
    "Rayo_Vallecano.csv",
    "Real_Betis.csv",
    "Real_Madrid.csv",
    "Real_Sociedad.csv",
    "Real_Valladolid.csv",
    "Sevilla.csv",
    "Valencia.csv",
    "Villarreal.csv",
]

In [4]:
def analyze_team_shots(file_path):
    df = pd.read_csv(file_path)
    
    total_seqs = len(df)
    shot_mask = df['words'].str.contains('Shot')
    shot_df = df[shot_mask].copy()
    num_shots = len(shot_df)
    
    if num_shots == 0:
        return {
            'Team': df['team_name'].iloc[0] if not df.empty else file_path,
            'Total_Sequences': total_seqs,
            'Shot_Count': 0,
            'Shot_Rate_%': 0,
            'Avg_Shot_Length': 0,
            'Signature_Shot_Sequence': "N/A",
            'Avg_K_Shot': 0,
            'Avg_Beta_Shot': 0
        }

    # Calcoliamo la lunghezza di ogni sequenza
    shot_df['len'] = shot_df['words'].apply(lambda x: len(x.split('-')))
    
    # --- MODIFICA PER LA SIGNATURE ---
    # Filtriamo le sequenze per trovare la "vera" firma tattica:
    # Cerchiamo la sequenza più comune che abbia almeno 3 azioni (es. Pass-Carry-Shot)
    elaborated_shots = shot_df[shot_df['len'] >= 3]
    
    if not elaborated_shots.empty:
        signature = elaborated_shots['words'].value_counts().idxmax()
    else:
        # Se la squadra non ha mai sequenze da 3+ azioni (raro), prendiamo la più frequente in assoluto
        signature = shot_df['words'].value_counts().idxmax()
    # ---------------------------------
    
    return {
        'Team': df['team_name'].iloc[0],
        'Total_Sequences': total_seqs,
        'Shot_Count': num_shots,
        'Shot_Rate_%': (num_shots / total_seqs) * 100,
        'Avg_Shot_Length': shot_df['len'].mean(),
        'Signature_Shot_Sequence': signature,
        'Avg_K_Shot': shot_df['K'].mean(),
        'Avg_Beta_Shot': shot_df['beta'].mean()
    }

# Il resto del codice rimane invariato
available_files = [f for f in team_files if os.path.exists(f)]
results = []
for f in available_files:
    results.append(analyze_team_shots(f))

summary_df = pd.DataFrame(results)

In [5]:
summary_df

,Team,Total_Sequences,Shot_Count,Shot_Rate_%,Avg_Shot_Length,Signature_Shot_Sequence,Avg_K_Shot,Avg_Beta_Shot
0,Athletic Club,20467,470,2.296380,3.400000,Ball Recovery-Carry-Shot,1.491360,0.745412
1,Atlético Madrid,19139,474,2.476618,3.542194,Pass-Ball Receipt-Shot,1.351397,0.779379
2,Barcelona,19389,677,3.491671,4.054653,Pass-Ball Receipt-Shot,1.439234,0.777955
3,Celta Vigo,19835,453,2.283842,3.779249,Pass-Ball Receipt-Shot,1.324015,0.775045
4,Deportivo Alavés,19974,410,2.052668,3.126829,Pass-Ball Receipt-Shot,1.356295,0.755792
5,Espanyol,19018,355,1.866653,3.591549,Pass-Ball Receipt-Carry-Shot,1.279780,0.770986
6,Getafe,20229,435,2.150378,2.733333,Ball Recovery-Carry-Shot,1.471249,0.734698
7,Girona,19192,408,2.125886,4.306373,Pass-Ball Receipt-Shot,1.469791,0.750032
8,Las Palmas,19585,413,2.108757,3.375303,Pass-Ball Receipt-Shot,1.424851,0.757349
9,Leganés,19430,353,1.816778,3.206799,Pass-Ball Receipt-Shot,1.431162,0.749540


In [6]:
# 1. Data collection from all available files
results = []
for file in team_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        # Filter sequences that lead to a shot
        shot_df = df[df['words'].str.contains('Shot')].copy()
        
        if not shot_df.empty:
            results.append({
                'Team': df['team_name'].iloc[0],
                'Shot_Rate_%': (len(shot_df) / len(df)) * 100,
                'Avg_K_Shot': shot_df['K'].mean()
            })

df_plot = pd.DataFrame(results)

# 2. Creation of the Interactive Scatter Plot
# Define the bubble chart
chart = alt.Chart(df_plot).mark_circle(size=250, opacity=0.8).encode(
    x=alt.X(
        'Shot_Rate_%',
        title='Efficiency (Shot Rate %)',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
        'Avg_K_Shot',
        title='Tactical Complexity (Avg K)',
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color('Team', legend=None),  # Distinct colors, legend hidden for clarity
    tooltip=[
        alt.Tooltip('Team', title='Team'),
        alt.Tooltip('Shot_Rate_%', title='Efficiency %', format='.2f'),
        alt.Tooltip('Avg_K_Shot', title='Complexity (K)', format='.4f')
    ]
).properties(
    title="Offensive Language Map: Efficiency vs Complexity",
    width=800,
    height=500
)

# 3. Add labels (team names)
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=12,
    fontSize=11
).encode(
    text='Team'
)

# 4. Mean lines (to identify the four quadrants)
line_x = alt.Chart(
    pd.DataFrame({'x': [df_plot['Shot_Rate_%'].mean()]})
).mark_rule(
    color='red',
    strokeDash=[3, 3]
).encode(x='x')

line_y = alt.Chart(
    pd.DataFrame({'y': [df_plot['Avg_K_Shot'].mean()]})
).mark_rule(
    color='blue',
    strokeDash=[3, 3]
).encode(y='y')

# 5. Final interactive visualization
final_chart = (chart + text + line_x + line_y).interactive()
final_chart.display()

alt.LayerChart(...)

In [7]:
# 1. Data collection from all available files
results = []
for file in team_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        # Filter sequences that lead to a shot
        shot_df = df[df['words'].str.contains('Shot')].copy()
        
        if not shot_df.empty:
            # Calculate sequence length (number of actions) for each shot sequence
            shot_df['len'] = shot_df['words'].apply(lambda x: len(x.split('-')))
            
            results.append({
                'Team': df['team_name'].iloc[0],
                'Avg_Shot_Length': shot_df['len'].mean(),
                'Avg_K_Shot': shot_df['K'].mean()
            })

df_plot = pd.DataFrame(results)

# 2. Interactive Scatter Plot Creation with Altair
chart = alt.Chart(df_plot).mark_circle(size=250, opacity=0.8).encode(
    x=alt.X('Avg_Shot_Length', 
            title='Average Shot Sequence Length (Avg_Shot_Length)', 
            scale=alt.Scale(zero=False)),
    y=alt.Y('Avg_K_Shot', 
            title='Tactical Complexity (Avg_K_Shot)', 
            scale=alt.Scale(zero=False)),
    color=alt.Color('Team', legend=None), # Different color for each team
    tooltip=[
        alt.Tooltip('Team', title='Team'),
        alt.Tooltip('Avg_Shot_Length', title='Avg Length', format='.2f'),
        alt.Tooltip('Avg_K_Shot', title='Complexity (K)', format='.4f')
    ]
).properties(
    title="Offensive Language Analysis: Build-up vs. Variety",
    width=800,
    height=500
)

# 3. Adding team names next to the data points
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=12,
    fontSize=11
).encode(
    text='Team'
)

# 4. Adding average reference lines (Quadrants)
line_x = alt.Chart(pd.DataFrame({'x': [df_plot['Avg_Shot_Length'].mean()]})).mark_rule(
    color='red', strokeDash=[3, 3], size=1
).encode(x='x')

line_y = alt.Chart(pd.DataFrame({'y': [df_plot['Avg_K_Shot'].mean()]})).mark_rule(
    color='blue', strokeDash=[3, 3], size=1
).encode(y='y')

# 5. Final interactive visualization (Zoom + Tooltip)
final_chart = (chart + text + line_x + line_y).interactive()
final_chart.display()

alt.LayerChart(...)